In [1]:
import pandas as pd
import numpy as np
import os 

In [2]:
import PineBioML
from PineBioML.preprocessing import IO

In [3]:
os.listdir(".")

['.git',
 '.ipynb_checkpoints',
 'ablation.ipynb',
 'documents',
 'example_feature_selection copy.ipynb',
 'example_feature_selection.ipynb',
 'images',
 'input',
 'LICENSE',
 'log.txt',
 'output',
 'package',
 'Pipfile',
 'Pipfile.lock',
 'README.md',
 'requirements.txt']

# IO

In [4]:
input_path = "./input/proteomics.csv"
export_title = "PDA "

In [5]:
proteomics = IO.read_file(input_path).set_index("patientID-radiomics", drop = True).drop("Withdrawn", axis =1)
proteomics.head(5)

,Tumor/Normal,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
patientID-radiomics,,,,,,,,,,,,,,,,,,,,,
C3N-03884,Tumor,28.424135,24.246165,29.366812,NaN,NaN,NaN,22.839828,24.662698,23.246860,...,NaN,25.017461,24.306182,NaN,NaN,18.946509,21.198617,28.930222,25.255542,NaN
C3L-00589,Normal Adjacent Tissue,28.502926,25.717743,29.883149,NaN,NaN,NaN,23.046828,25.270276,25.790888,...,NaN,25.295479,24.091819,NaN,NaN,18.750975,20.866134,28.242476,26.168199,NaN
C3L-03123,Normal Adjacent Tissue,28.297751,24.580732,29.130757,NaN,NaN,NaN,22.625520,24.676915,23.268602,...,NaN,25.186219,23.137290,NaN,NaN,18.304164,21.223762,28.269888,25.257447,NaN
C3L-01687,Normal Adjacent Tissue,28.662810,25.795589,29.768221,NaN,NaN,NaN,23.093415,24.957128,24.926253,...,NaN,25.188314,24.046851,NaN,NaN,18.644933,21.022914,28.310557,25.858360,NaN
C3L-03123,Tumor,28.481919,23.815981,29.085915,NaN,NaN,NaN,23.336621,24.811962,22.560420,...,NaN,25.037140,23.892358,NaN,NaN,18.513350,21.160126,28.869091,25.144300,NaN


In [6]:
x = np.power(2, proteomics.drop("Tumor/Normal ", axis = 1)).reset_index(drop = True)
y = (proteomics["Tumor/Normal "] == "Tumor").reset_index(drop = True)

In [7]:
x.head(5)

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADAT,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,3.601780e+08,1.989862e+07,6.922953e+08,NaN,NaN,NaN,7.507121e+06,2.655900e+07,9.954105e+06,NaN,...,NaN,3.396301e+07,2.074387e+07,NaN,NaN,505204.942515,2.406687e+06,5.115225e+08,4.005674e+07,NaN
1,3.803959e+08,5.518385e+07,9.902032e+08,NaN,NaN,NaN,8.665357e+06,4.046794e+07,5.805382e+07,NaN,...,NaN,4.118111e+07,1.787970e+07,NaN,NaN,441169.965734,1.911315e+06,3.175649e+08,7.540723e+07,NaN
2,3.299680e+08,2.509213e+07,5.878028e+08,NaN,NaN,NaN,6.470833e+06,2.682203e+07,1.010525e+07,NaN,...,NaN,3.817749e+07,9.226104e+06,NaN,NaN,323669.898241,2.449001e+06,3.236565e+08,4.010968e+07,NaN
3,4.249772e+08,5.824332e+07,9.143817e+08,NaN,NaN,NaN,8.949746e+06,3.257198e+07,3.188232e+07,NaN,...,NaN,3.823298e+07,1.733100e+07,NaN,NaN,409905.799064,2.130726e+06,3.329101e+08,6.083339e+07,NaN
4,3.748969e+08,1.476810e+07,5.698136e+08,NaN,NaN,NaN,1.059310e+07,2.945404e+07,6.185335e+06,NaN,...,NaN,3.442946e+07,1.557100e+07,NaN,NaN,374174.127860,2.343325e+06,4.903005e+08,3.708417e+07,NaN


In [8]:
y.head(5)

0     True
1    False
2    False
3    False
4     True
Name: Tumor/Normal , dtype: bool

# Preprocessing

### Impute

In [9]:
from PineBioML.preprocessing import impute, bagging

In [10]:
simple = impute.simple_imputer(threshold = 0.3333, strategy = "median")
simple.fit_transform(x, y)

knn = impute.knn_imputer(threshold = 0.3333, n_neighbor = 5)
x, y = knn.fit_transform(x, y)

In [11]:
x.head(5)

,A1BG,A1CF,A2M,A2ML1,AAAS,AACS,AADAC,AAGAB,AAK1,AAMDC,...,ZPR1,ZRANB2,ZSCAN18,ZSWIM8,ZW10,ZWILCH,ZYG11B,ZYX,ZZEF1,ZZZ3
0,3.601780e+08,1.989862e+07,6.922953e+08,1.001351e+07,7.507121e+06,2.655900e+07,9.954105e+06,1.700950e+07,3.930431e+07,5.692694e+07,...,3.962214e+07,1.290026e+08,6.101920e+06,6.596625e+06,3.396301e+07,2.074387e+07,2.406687e+06,5.115225e+08,4.005674e+07,1.358649e+06
1,3.803959e+08,5.518385e+07,9.902032e+08,2.428902e+06,8.665357e+06,4.046794e+07,5.805382e+07,1.643521e+07,3.500768e+07,8.411769e+07,...,3.034699e+07,1.077997e+08,5.956067e+06,6.544889e+06,4.118111e+07,1.787970e+07,1.911315e+06,3.175649e+08,7.540723e+07,1.027846e+06
2,3.299680e+08,2.509213e+07,5.878028e+08,3.556857e+06,6.470833e+06,2.682203e+07,1.010525e+07,1.625682e+07,3.763716e+07,8.809936e+07,...,3.010763e+07,1.482483e+08,5.617840e+06,6.430990e+06,3.817749e+07,9.226104e+06,2.449001e+06,3.236565e+08,4.010968e+07,1.449336e+06
3,4.249772e+08,5.824332e+07,9.143817e+08,2.428902e+06,8.949746e+06,3.257198e+07,3.188232e+07,1.644599e+07,3.502954e+07,7.932558e+07,...,3.443084e+07,1.074707e+08,9.190937e+06,6.714496e+06,3.823298e+07,1.733100e+07,2.130726e+06,3.329101e+08,6.083339e+07,1.027846e+06
4,3.748969e+08,1.476810e+07,5.698136e+08,5.521517e+06,1.059310e+07,2.945404e+07,6.185335e+06,1.853726e+07,4.363844e+07,5.637029e+07,...,4.181187e+07,1.342225e+08,6.403648e+06,6.687848e+06,3.442946e+07,1.557100e+07,2.343325e+06,4.903005e+08,3.708417e+07,1.299666e+06


### Bagging

bagger = bagging.bagger()
x, y = bagger.fit_transform(x, y)

print(bagger.see("AAMP")) # AAMP is a feature
print(bagger.see("AA"))   # no AA but some other feature start with AA
print(bagger.see("AA*"))  # * can list all features start with ZN
try:
    print(bagger.see("ERROR"))
except:
    print("ERROR is not a feature in x, it will raise an error")

# Feature selection

In [12]:
from PineBioML.selection.ensemble import selector

In [13]:
ensemble_selector = selector()
important_feature, importance = ensemble_selector.select(x, y, 20)

Using  pcRF_permutation  to select.
pc RF kernel fitting.


100%|██████████| 512/512 [00:03<00:00, 167.98it/s]


Permutation evaluation start! n_repeat:  5
    repeat:  0


  0%|          | 0/162 [00:00<?, ?it/s]c:\Users\aimhi\Documents\GitHub\undetermined\package\selection\RF.py:134: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.542016806722689' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  score.loc[col, i] = self.kernel.evaluate(x_permute, y, oob = True)
100%|██████████| 162/162 [02:07<00:00,  1.27it/s]


    repeat:  1


...

    repeat:  3


  0%|          | 0/162 [00:00<?, ?it/s]c:\Users\aimhi\Documents\GitHub\undetermined\package\selection\RF.py:134: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5546218487394958' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  score.loc[col, i] = self.kernel.evaluate(x_permute, y, oob = True)
100%|██████████| 162/162 [02:07<00:00,  1.27it/s]


    repeat:  4


  0%|          | 0/162 [00:00<?, ?it/s]c:\Users\aimhi\Documents\GitHub\undetermined\package\selection\RF.py:134: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5378151260504201' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  score.loc[col, i] = self.kernel.evaluate(x_permute, y, oob = True)
100%|██████████| 162/162 [02:08<00:00,  1.26it/s]

pcRF_permutation  is done.



# Output

# unbagging
important_feature = important_feature.map(bagger.unbagging) 

In [14]:
important_feature.to_csv("./output/"+export_title+ " important feature.csv")

In [15]:
important_feature

,pcRandomForest_permutation
0,PTTG1IP
1,RUVBL2
2,RPF2
3,GAR1
4,RBP7
5,SPN
6,MICALL1
7,GLTP
8,UBA6
9,AP4S1


In [16]:
importance.to_csv("./output/"+export_title+ " importance.csv")